In [ ]:
'''
    "항공권":[{"subMenu1": "항공권서브메뉴1 옵션2"}, {"subMenu2": "항공권서브메뉴2 옵션3"},{"subMenu3": "항공권서브메뉴3 옵션2"}],
    "렌트카":[{"subMenu1":"렌트카서브메뉴1 옵션2"},{"subMenu2":"렌트카서브메뉴2 옵션2"},{"subMenu3":"렌트카서브메뉴3 옵션1"}],
    "호텔":[{"subMenu1":"호텔서브메뉴1 옵션1"},{"subMenu2":"호텔서브메뉴2 옵션3"},{"subMenu3":"호텔서브메뉴3 옵션1"}]

{"항공권":[{"subMenuKey":"sub-menu 항공권서브 메뉴1","subMenuValue":"항공권서브 메뉴1옵션1"},{"subMenuKey":"sub-menu 항공권서브 메뉴2","subMenuValue":"항공권서브 메뉴2옵션2"}],
"렌트카":[{"subMenuKey":"sub-menu 렌트카서브 메뉴1","subMenuValue":"렌트카서브 메뉴1옵션2"}],
"호텔":[{"subMenuKey":"sub-menu 호텔서브 메뉴2","subMenuValue":"호텔서브 메뉴2옵션3"}],
"dateRange":"07/25/2023 - 07/25/2023"}


'''

In [4]:
json_data = {
    "렌트카":[{"start_date": "2023-01-01"}, {"end_date": "2023-01-10"}],
    "항공권":[{"departure_datetime": "2023-01-01"}, {"arrival_datetime": "2023-01-10"}],
    "호텔":[{"start_date": "2023-01-01"}, {"end_date": "2023-01-10"}]
}
date_keys = [list(item.keys())[0] for item in json_data["렌트카"]]
date_keys2 = [list(item.keys())[0] for item in json_data["호텔"]]
date_keys3 = [list(item.keys())[0] for item in json_data["항공권"]]

# 1. 렌트카
sql_query = f"""
SELECT *
FROM cars
WHERE
  seats = :selected_seats AND
  size = :selected_size AND
  transmission_type = :selected_transmission_type AND
  age_req = :selected_age_req AND
  driving_experice = :selected_driving_experience AND
  금전상황 = :selected_금전상황 AND
  {date_keys[0]} = :selected_start_date AND
  {date_keys[1]} = :selected_end_date
ORDER BY ratings DESC
LIMIT 3
"""

# 2. 호텔
sql_query2 = f"""
SELECT *
FROM hotels
WHERE
  region = :selected_region AND
  capacity = :selected_capacity AND
  금전상황 = :selected_금전상황 AND
  {date_keys2[0]} = :selected_start_date AND
  {date_keys2[1]} = :selected_end_date
ORDER BY ratings DESC
LIMIT 3
"""


# 3. 항공권 
sql_query3 = f"""
SELECT 
    sub.name, 
    sub.seat, 
    sub.adult_charge, 
    sub.leavehour, 
    sub.departure, 
    sub.arrival, 
    sub.departure_datetime, 
    sub.arrival_datetime, 
    sub.check, 
    sub.child_charge, 
    sub.departure_kor, 
    sub.arrival_kor,
    sub.adult_charge * :num_adults + sub.child_charge * :num_children AS total_price
FROM 
    (SELECT 
        flights.name, 
        flights.seat, 
        flights.adult_charge, 
        flights.leavehour, 
        flights.departure, 
        flights.arrival, 
        flights.departure_datetime, 
        flights.arrival_datetime, 
        flights.check, 
        flights.child_charge, 
        flights.departure_kor, 
        flights.arrival_kor
    FROM 
        flights 
    WHERE 
        flights.departure_kor = :departure_kor AND
        flights.arrival_kor = :arrival_kor AND
        flights.금전상황 = :금전상황 AND
        {date_keys3[0]} = :departure_datetime AND
        {date_keys3[1]} = :arrival_datetime
    ORDER BY 
        flights.adult_charge 
    LIMIT 3) AS sub
"""


In [5]:
def update_json_dates(json_data, selected_dates):
    for category in json_data:
        for i, item in enumerate(json_data[category]):
            for key in item:
                # If the key is a date key, update the value with the selected date
                if 'date' in key or 'datetime' in key:
                    json_data[category][i][key] = selected_dates[key]

selected_dates = {
    "start_date": "2023-01-05",
    "end_date": "2023-01-15",
    "departure_datetime": "2023-01-05",
    "arrival_datetime": "2023-01-15"
}
update_json_dates(json_data, selected_dates)

print(json_data)


{'렌트카': [{'start_date': '2023-01-05'}, {'end_date': '2023-01-15'}], '항공권': [{'departure_datetime': '2023-01-05'}, {'arrival_datetime': '2023-01-15'}], '호텔': [{'start_date': '2023-01-05'}, {'end_date': '2023-01-15'}]}


In [ ]:
'''
< 렌트카 SQL >
- cars 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :selected_seats, :selected_size, :selected_transmission_type, :selected_age_req, :selected_driving_experience, :selected_choice, :selected_start_date, :selected_end_date 은 사용자가 선택해야할 값

SELECT *
FROM cars
WHERE
  seats = :selected_seats AND
  size = :selected_size AND
  transmission_type = :selected_transmission_type AND
  age_req = :selected_age_req AND
  driving_experice = :selected_driving_experience AND
  choice = :selected_choice AND
  start_date = :selected_start_date AND
  end_date = :selected_end_date
ORDER BY ratings DESC
LIMIT 3
'''

In [ ]:
''' 
< 호텔 SQL >
- hotels 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :selected_region, :selected_capacity, :selected, choice 은 사용자가 선택해야할 값

SELECT *
FROM 
    hotels 
INNER JOIN 
    cars ON DATE(hotels.start_date) = DATE(cars.start_date) AND DATE(hotels.end_date) = DATE(cars.end_date)
WHERE 
    hotels.region = :selected_region AND
    hotels.capacity = :selected_capacity AND
    hotels.choice = :selected choice 
ORDER BY 
    hotels.ratings DESC 
LIMIT 3
'''

In [ ]:
'''
< 항공권 SQL >
- flights 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :num_adults, :num_children, :departure_kor, :arrival_kor, choice 은 사용자가 선택해야할 값


SELECT 
    sub.name, 
    sub.seat, 
    sub.adult_charge, 
    sub.leavehour, 
    sub.departure, 
    sub.arrival, 
    sub.departure_datetime, 
    sub.arrival_datetime, 
    sub.check, 
    sub.child_charge, 
    sub.departure_kor, 
    sub.arrival_kor,
    sub.adult_charge * :num_adults + sub.child_charge * :num_children AS total_price
FROM 
    (SELECT 
        flights.name, 
        flights.seat, 
        flights.adult_charge, 
        flights.leavehour, 
        flights.departure, 
        flights.arrival, 
        flights.departure_datetime, 
        flights.arrival_datetime, 
        flights.check, 
        flights.child_charge, 
        flights.departure_kor, 
        flights.arrival_kor
    FROM 
        flights 
    INNER JOIN 
        cars ON DATE(flights.departure_datetime) = DATE(cars.start_date) AND DATE(flights.arrival_datetime) = DATE(cars.end_date)
    WHERE 
        flights.departure_kor = :departure_kor AND
        flights.arrival_kor = :arrival_kor AND
        flights.choice = :choice 
    ORDER BY 
        flights.adult_charge 
    LIMIT 3) AS sub
'''